In [1]:
from huggingface_hub import login
login()

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch
device = "cuda" # the device to load the model onto

model_name = "Qwen/Qwen1.5-MoE-A2.7B"

# 4-bit QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 # QLoRA = 4-bit base model
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",         # QLoRA uses nf4
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model.eval()
inputs = tokenizer("Hello!", return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(out[0], skip_special_tokens=True))

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from peft import IA3Config, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

config = LoraConfig(r = 4,
                    lora_alpha=4,
                    target_modules = ["gate", "gate_proj", "up_proj", "down_proj"],
                    lora_dropout=0.1
                    )

lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Hello! I'm a 20 year old girl from the Netherlands. I'm a student and I'm
trainable params: 62,133,888 || all params: 14,377,918,080 || trainable%: 0.4321


In [3]:
# from datasets import load_dataset

# dataset = load_dataset("Na0s/sft-ready-Text-Generation-Augmented-Data", split="train")

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/22 [00:00<?, ?it/s]

In [4]:
# print(dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 7667416
})


In [5]:
# def tokenize(batch):
#     texts = []
#     for prompt, completion in zip(batch["prompt"], batch["completion"]):
#         text = tokenizer.apply_chat_template(
#             [
#                 {"role": "user", "content": prompt},
#                 {"role": "assistant", "content": completion},
#             ],
#             tokenize=False,
#             add_generation_prompt=False,
#         )

#         if isinstance(text, list):
#             text = "".join(text)

#         texts.append(text)

#     return tokenizer(
#         texts,
#         truncation=True,
#         padding=False,
#         max_length=2048
#     )

# batch = dataset.select(range(3))
# tokens = tokenize(batch)
# tokenizer.decode(tokens["input_ids"][0])


In [6]:
# tokenized_dataset = dataset.map(
#     tokenize,
#     batched=True,
#     num_proc=75,
#     remove_columns=["prompt", "completion"],
# )

# print(tokenized_dataset[0])

NameError: name 'tokenized_dataset' is not defined

In [5]:
from datasets import load_dataset
tokenized_dataset = load_dataset("kaaiiii/tokenized_data_for_MoE", split="train")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [8]:
EOS = tokenizer.eos_token_id  # Qwen: tokenizer.eos_token_id

def add_eos(batch):
    new_input_ids = []
    new_attention = []

    for ids, mask in zip(batch["input_ids"], batch["attention_mask"]):
        ids = ids + [EOS]
        mask = mask + [1]
        new_input_ids.append(ids)
        new_attention.append(mask)

    return {
        "input_ids": new_input_ids,
        "attention_mask": new_attention,
    }

tokenized_dataset = tokenized_dataset.map(add_eos, batched=True, num_proc=75)

In [ ]:
# print(tokenized_dataset[0:5])
# tokenized_dataset.save_to_disk("./tokenized_dataset")

In [9]:

tokenizer.decode(tokenized_dataset["input_ids"][0])

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n0.002 = 1000 \n1 = x?<|im_end|>\n<|im_start|>assistant\nTo find the value of x, we can set up a proportion using the given information:\n\n0.002/1000 = 1/x\n\nTo solve for x, we can cross multiply:\n\n0.002 * x = 1000 * 1\n\n0.002x = 1000\n\nDividing both sides by 0.002:\n\nx = 1000 / 0.002\n\nx = 500,000\n\nTherefore, 1 is equal to 500,000 in this proportion.<|im_end|>\n<|endoftext|>'

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM
)

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

lora_model.enable_input_require_grads()
lora_model.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="./outputs_endoftext",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8, 
    max_steps=2000,               # try a small number first

    dataloader_num_workers = 4,
    dataloader_pin_memory=True,
    dataloader_prefetch_factor=4,
    dataloader_persistent_workers=True,
    
    learning_rate=1e-6,
    weight_decay=0.03,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    bf16=True,

    logging_steps=10,
    logging_strategy="steps",
    logging_first_step=True,
    disable_tqdm=False,    # ensure progress bar is visible

    save_strategy="steps",
    save_steps=200,
    save_total_limit=10,
    save_safetensors=True,
    push_to_hub=True,
    hub_model_id="kaaiiii/QwenA2.7B_endoftext",
)

trainer = Trainer(
    model=lora_model,          # your LoRA-wrapped model
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

trainer.model.save_pretrained("Qwen1.5_QwenA2.7B_endoftext")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,2.001500
10,1.789100
20,1.878900
30,1.810600
40,1.740800
50,1.823100
60,1.768000
70,1.891400
80,1.806200
90,1.890700


In [ ]:
# trainer.model.push_to_hub("kaaiiii/Qwen1.5b_MoE_LoRA_mode_500_steps")

In [ ]:
# from trl import SFTConfig
# import logging
# logging.basicConfig(level=logging.ERROR)

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-MoE-A2.7B", use_fast=True)
# print(lora_model.device)

# lora_model.enable_input_require_grads()
# lora_model.gradient_checkpointing_enable()

# trainer = SFTTrainer(
#     model = lora_model,
#     train_dataset = dataset,
#     processing_class = tokenizer,
#     args = SFTConfig(
#         per_device_train_batch_size = 4,
#         gradient_accumulation_steps = 8,

#         dataloader_num_workers = 8,
#         dataloader_pin_memory=True,
#         dataloader_prefetch_factor=4,
#         dataloader_persistent_workers=True,
        
#         packing = True,
#         group_by_length = True,
#         warmup_ratio = 0.05,
#         bf16 = True,
#         max_steps=1000,
#         learning_rate = 5e-6,
#         optim = "adamw_8bit",
#         weight_decay = 0.03,
#         lr_scheduler_type = "cosine",
#         seed = 423,
#         eval_strategy="no",
#         do_eval=False,
#         output_dir = "./outputs_SFT",

#         save_strategy="steps",
#         save_steps=500,
#         save_total_limit=10,
#         save_safetensors=True,
#         push_to_hub=True,   
#         hub_model_id="kaaiiii/Qwen1.5_MoE_2000steps",
        
#         remove_unused_columns=False,
#         logging_steps = 10,
#         disable_tqdm=False,
#         report_to="none"
#     )
# )

# torch.cuda.empty_cache()

# trainer.train()

# trainer.model.save_pretrained("Qwen1.5_MoE_lora_model_lr5e_6_1000steps")

In [ ]:
print(trainer.model)

In [ ]:
print(model)

In [ ]:
print(lora_model)

In [ ]:
def find_all_router_layers(model):
    router_layers = []
    for name, module in model.named_modules():
        if "mlp.gate" in name or "mlp.shared_expert_gate" in name:
            router_layers.append(name)
    return router_layers

routers = find_all_router_layers(model)
for r in routers:
    print(r)

In [ ]:
def get_router_l2_norms(model):
    layer_norms = {}

    for name, module in model.named_modules():
        if "mlp.gate" in name and hasattr(module, "base_layer"):
            weight = module.base_layer.weight  # (60, 2048)
            norms = torch.norm(weight.float(), dim=1)  # shape (60,)
            layer_norms[name] = norms.detach().cpu()

    return layer_norms

router_norms = get_router_l2_norms(model)

for name, norms in list(router_norms.items())[:3]:
    print(norms.shape)
    print(name, norms[:10])

In [ ]:
print(router_norms)

In [ ]:
import matplotlib.pyplot as plt

def plot_all_layers(norms):
    plt.figure(figsize=(12, 7))

    for i, (name, norm) in enumerate(norms.items()):
        plt.plot(norm, label=f"L{i}", alpha=0.5)

    plt.title("Router Expert L2 Norms Across All Layers")
    plt.xlabel("Expert Index (0-59)")
    plt.ylabel("L2 Norm")
    plt.legend(ncol=3, fontsize=7)
    plt.grid(True)
    plt.show()

plot_all_layers(router_norms)

In [ ]:
def choose_experts_to_prune(router_norms, prune_ratio=0.1):
    prune_indices = {}

    for layer_name, norms in router_norms.items():
        k = int(len(norms) * prune_ratio)
        prune_idx = torch.topk(norms, k=k, largest=False).indices
        prune_indices[layer_name] = prune_idx.tolist()

    return prune_indices

prune_plan = choose_experts_to_prune(router_norms, prune_ratio=0.1)

In [ ]:
def prune_moe_layer(mlp, prune_idx):
    prune_idx = sorted(prune_idx)

    # 1. prune gate
    old_w = mlp.gate.base_layer.weight  # (60, 2048)
    keep_idx = [i for i in range(old_w.size(0)) if i not in prune_idx]

    new_gate_weight = old_w[keep_idx, :].clone()
    mlp.gate.base_layer.weight = torch.nn.Parameter(new_gate_weight)

    # 2. prune experts
    new_experts = torch.nn.ModuleList([
        exp for i, exp in enumerate(mlp.experts)
        if i not in prune_idx
    ])
    mlp.experts = new_experts

    print(f"Pruned {len(prune_idx)} experts, left {len(mlp.experts)}.")

In [ ]:
def prune_model(model, prune_plan):
    for name, module in model.named_modules():
        if "mlp" in name and hasattr(module, "experts"):
            if name + ".gate" in prune_plan:
                prune_idx = prune_plan[name + ".gate"]
                prune_moe_layer(module, prune_idx)